In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import verde as vd
from datetime import datetime


In [3]:
# Función para guardar los resultados
def guardar_resultados(grid, grid_temperatura, fecha_objetivo, region, latitudes, longitudes, temperaturas, colombia_shapefile):
    # Crear las carpetas si no existen
    output_npy_dir = 'data/tmean_interp_final/npy'
    output_png_dir = 'data/tmean_interp_final/png'
    os.makedirs(output_npy_dir, exist_ok=True)
    os.makedirs(output_png_dir, exist_ok=True)

    # Guardar la grilla de temperaturas como un archivo .npy
    npy_file_path = os.path.join(output_npy_dir, f'temperatura_{fecha_objetivo}.npy')
    np.save(npy_file_path, grid_temperatura)

    # Cargar el archivo de límites administrativos de Colombia con Geopandas
    gdf = gpd.read_file(colombia_shapefile)

    # Reproyectar los datos geográficos a WGS84 (EPSG:4326)
    if gdf.crs != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")

    # Filtrar para mostrar solo el departamento de Santander
    santander_gdf = gdf[gdf['DeNombre'] == 'Santander']  # Asegúrate de que coincida con el nombre en el archivo

    # Graficar el mapa de temperatura interpolado
    plt.figure(figsize=(10, 8))
    plt.pcolormesh(grid[0], grid[1], grid_temperatura, cmap='coolwarm', shading='auto')
    plt.colorbar(label='Temperatura (°C)')

    # Graficar los puntos de datos originales
    plt.scatter(longitudes, latitudes, c=temperaturas, s=10, edgecolor='k', cmap='coolwarm', label='Datos originales')

    # Superponer los límites geográficos de los municipios de Santander
    santander_gdf.boundary.plot(ax=plt.gca(), linewidth=1, edgecolor="black", label="Límites de Santander")

    # Limitar los ejes al área de interés
    plt.xlim(region[0], region[1])
    plt.ylim(region[2], region[3])

    # Etiquetas y título
    plt.xlabel('Longitud')
    plt.ylabel('Latitud')
    plt.title(f'Mapa denso de temperatura (Interpolación) para la fecha {fecha_objetivo}')
    plt.legend()

    # Guardar el gráfico como PNG
    png_file_path = os.path.join(output_png_dir, f'temperatura_{fecha_objetivo}.png')
    plt.savefig(png_file_path, dpi=300)
    plt.close()

# Leer los datos desde un archivo Excel
data = pd.read_excel('data/tmean.xlsx')
data['Fecha'] = pd.to_datetime(data['Fecha'])

# Definir la región de interés
region = ((-74.6), (-72.4), (5.5), (8.2))  # Latitudes y longitudes de la región de interés
spacing = 0.01  # Espaciado entre los puntos de la malla

# Obtener los años disponibles en los datos
años_disponibles = data['Fecha'].dt.year.unique()

# Iterar sobre todos los años disponibles
for anio in años_disponibles:
    print(f"Procesando datos para el año {anio}...")

    # Iterar sobre los meses de cada año
    for mes in range(1, 13):
        # Filtrar los datos para el primer día de cada mes
        fecha_objetivo = datetime(anio, mes, 1)
        data_fecha = data[data['Fecha'] == fecha_objetivo]

        if data_fecha.empty:
            print(f"No se encontraron datos para la fecha {fecha_objetivo}")
            continue

        # Extraer las columnas relevantes
        latitudes = data_fecha['Latitud'].values
        longitudes = data_fecha['Longitud'].values
        temperaturas = data_fecha['Valor_medio'].values

        # Crear una malla densa para la interpolación
        grid = vd.grid_coordinates(region, spacing=spacing)

        # Interpolación usando Verde Spline
        interp = vd.Spline()  # Configurar el interpolador Spline
        interp.fit((longitudes, latitudes), temperaturas)  # Ajustar el interpolador a los datos

        # Predecir valores en la malla densa
        grid_temperatura = interp.predict(grid)

        # Guardar los resultados: grilla en .npy y gráfico en .png
        guardar_resultados(grid, grid_temperatura, fecha_objetivo.strftime('%Y-%m-%d'), region, latitudes, longitudes, temperaturas, 'data/aoi/Departamento.shp')

print("Proceso completado para todos los años y meses.")


Procesando datos para el año 2000...
Procesando datos para el año 2001...
Procesando datos para el año 2002...
Procesando datos para el año 2003...
Procesando datos para el año 2004...
Procesando datos para el año 2005...
Procesando datos para el año 2006...
Procesando datos para el año 2007...
Procesando datos para el año 2008...
Procesando datos para el año 2009...
Procesando datos para el año 2010...
Procesando datos para el año 2011...
Procesando datos para el año 2012...
Procesando datos para el año 2013...


C:\Users\LUISRODRIGUEZ\AppData\Local\Temp\ipykernel_13216\661521408.py:46: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig(png_file_path, dpi=300)


Procesando datos para el año 2014...
Procesando datos para el año 2015...
Procesando datos para el año 2016...
Procesando datos para el año 2018...


C:\Users\LUISRODRIGUEZ\AppData\Local\Temp\ipykernel_13216\661521408.py:46: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig(png_file_path, dpi=300)


Procesando datos para el año 2019...
Procesando datos para el año 2017...
Procesando datos para el año 2022...
Procesando datos para el año 2023...


C:\Users\LUISRODRIGUEZ\AppData\Local\Temp\ipykernel_13216\661521408.py:46: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig(png_file_path, dpi=300)


Procesando datos para el año 2024...
No se encontraron datos para la fecha 2024-11-01 00:00:00
No se encontraron datos para la fecha 2024-12-01 00:00:00
Procesando datos para el año 2020...
Procesando datos para el año 2021...
Proceso completado para todos los años y meses.
